# Downsample uwb data to 1hz (convert to function)

In [5]:
import sqlite3
import pandas as pd
import numpy as np
import field_neuro as fn
import csv
from datetime import datetime

# Load your metadata sheet (if not already loaded)
meta = pd.read_excel(r'Y:\Data\FieldProject\FieldMission5\metadata.xlsx')  # Adjust the path to your metadata file

# Manually define each animals tag shortid, tag start time, and tag end time (or suspected death time) to solve the tag switching across animals issue. 
# information derived from my lab notebook notes and metadata. 
# note that this should never happen again, and should be avoided at all costs. Ideally, one UWB tag goes with one animal for the entire experiment. 

data = [
    ('MFEL', '3012', '2024-06-22 19:00:00', '2024-06-27 12:00:00'), ## estimated. Note first day of data lost
    ('MFEL', '304c', '2024-06-22 12:00:00', '2024-06-30 22:36:00'), ## estimated
    ('MFEL', '304d', '2024-06-30 22:36:00', '2024-07-03 17:36:00'), 
    ('MTAM', '3015', '2024-06-22 19:00:00', '2024-06-27 22:50:00'),
    ('MTAM', '3040', '2024-06-27 22:50:00', '2024-07-01 22:53:00'),
    ('MTAM', '303f', '2024-07-01 22:53:00', '2024-07-04 09:00:00'), ## caught by zipple, no headcap/tag
    ('MDAN', '3020', '2024-06-22 19:00:00', '2024-06-27 20:20:00'),
    ('MDAN', '303d', '2024-06-27 20:20:00', '2024-07-02 22:53:00'),
    ('MDAN', '3043', '2024-07-02 22:53:00', '2024-07-03 20:52:00'), ## handtrapped in tunnel under Z4
    ('MARV', '3019', '2024-06-22 19:00:00', '2024-06-25 17:40:00'),
    ('MARV', '3028', '2024-06-25 17:40:00', '2024-06-27 20:05:00'),
    ('MARV', '303f', '2024-06-27 20:05:00', '2024-06-30 22:20:00'), ## found dead under Z3
    ('MMTS', '3016', '2024-06-22 19:00:00', '2024-06-27 20:45:00'),
    ('MMTS', '3046', '2024-06-27 20:45:00', '2024-07-01 01:24:00'),
    ('MMTS', '303c', '2024-07-01 01:24:00', '2024-07-03 17:34:00'), ## trapped under z5
    ('MMTO', '3011', '2024-06-22 19:00:00', '2024-06-25 16:10:00'),
    ('MMTO', '3038', '2024-06-25 16:10:00', '2024-06-27 21:00:00'),
    ('MMTO', '3041', '2024-06-27 21:00:00', '2024-07-01 22:20:00'),
    ('MMTO', '3042', '2024-07-01 22:20:00', '2024-07-03 17:34:00'), ## trapped under z6, no baseplate, headcap, or microtag
    ('MRUF', '301e', '2024-06-22 19:00:00', '2024-07-27 22:10:00'),
    ('MRUF', '304e', '2024-06-27 22:10:00', '2024-07-03 20:30:00'), ## handtrapped in grass between anchor 12 and 15
    ('MMTN', '301c', '2024-06-22 19:00:00', '2024-06-27 22:36:00'),
    ('MMTN', '3044', '2024-06-27 22:36:00', '2024-07-01 02:07:00'),
    ('MMTN', '3043', '2024-07-01 02:07:00', '2024-07-01 21:00:00'), ## sacced during battery swap
    ('FROS', '3010', '2024-06-22 22:00:00', '2024-06-27 20:40:00'),
    ('FROS', '304a', '2024-06-27 20:40:00', '2024-07-01 01:38:00'),
    ('FROS', '3049', '2024-07-01 01:38:00', '2024-07-03 17:17:00'), ## trapped in z5
    ('FAUM', '3014', '2024-06-22 22:00:00', '2024-06-25 17:40:00'),
    ('FAUM', '3033', '2024-06-25 17:40:00', '2024-06-27 21:30:00'),
    ('FAUM', '304b', '2024-06-27 21:30:00', '2024-07-03 20:43:00'), ## trapped under z3
    ('FSBR', '300d', '2024-06-22 22:00:00', '2024-06-27 22:22:00'),
    ('FSBR', '304f', '2024-06-27 22:22:00', '2024-07-03 18:43:00'), ## trapped in z11
    ('FMRT', '3013', '2024-06-22 22:00:00', '2024-06-25 18:30:00'),
    ('FMRT', '302d', '2024-06-25 18:30:00', '2024-06-27 20:30:00'),
    ('FMRT', '3047', '2024-06-27 20:30:00', '2024-07-03 21:00:00'), ## trapped near anchor 22 along wall
    ('FMAR', '3018', '2024-06-22 22:00:00', '2024-06-27 16:10:00'),
    ('FMAR', '3037', '2024-06-27 16:10:00', '2024-06-27 20:53:00'),
    ('FMAR', '3045', '2024-06-27 20:53:00', '2024-07-03 17:50:00'), ## trapped in z6
    ('FBRY', '301d', '2024-06-22 22:00:00', '2024-06-25 16:10:00'),
    ('FBRY', '303a', '2024-06-25 16:10:00', '2024-06-27 21:05:00'),
    ('FBRY', '3048', '2024-06-27 21:05:00', '2024-07-03 17:43:00'), ## trapped in z6
    ('FTSS', '3021', '2024-06-22 22:00:00', '2024-06-30 22:20:00'), ## found dead under z3
    ('FCRN', '301f', '2024-06-22 22:00:00', '2024-06-27 21:55:00'),
    ('FCRN', '303e', '2024-06-27 21:55:00', '2024-07-03 18:22:00'), ## trapped in z8
]

# Create the DataFrame
meta_code_hex = pd.DataFrame(data, columns=['code', 'hex_id', 'tag_start_time', 'tag_end_time'])

# Confirm that tag_start_time and tag_end_time are timezone-naive
meta_code_hex['tag_start_time'] = pd.to_datetime(meta_code_hex['tag_start_time'])
meta_code_hex['tag_end_time'] = pd.to_datetime(meta_code_hex['tag_end_time'])

# Check for any rows where tag_start_time is greater than tag_end_time
invalid_intervals = meta_code_hex[meta_code_hex['tag_start_time'] > meta_code_hex['tag_end_time']]

# Display any invalid intervals
# print("Invalid intervals (start time > end time):")
# print(invalid_intervals)

# Display the DataFrame
# meta_code_hex


### view the data frame, optional
# Connect to the SQLite database, doesnt require loading entire dataset into RAM
# conn = sqlite3.connect(r"Y:\Data\FieldProject\FieldMission5\uwb\T005_uwb.db")

# # List all tables in the database
# tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
# print("Tables in the database:", tables)

# # Describe the structure of a specific table
# table_name = 'bravo_sql_databse_2024_CCVBehaviorTrial'
# table_info = pd.read_sql_query(f"PRAGMA table_info({table_name});", conn)
# print(f"Structure of the table {table_name}:", table_info)

# # View the first few rows of the table
# sample_data = pd.read_sql_query(f"SELECT * FROM {table_name} LIMIT 5;", conn)

# print(sample_data)

# # Get the total number of rows in the table; takes a very long time to run
# # row_count = pd.read_sql_query(f"SELECT COUNT(*) FROM {table_name};", conn)
# # print(f"Total number of rows in the table {table_name}: {row_count.iloc[0, 0]}")

# # Close the connection
# conn.close()

# All chunk testing

In [6]:
import pandas as pd
import numpy as np
import sqlite3
import os
import glob

# Connect to the SQLite database, doesn't require loading the entire dataset into RAM. Database is approximately 84 million rows worth of data
conn = sqlite3.connect(r"Y:\Data\FieldProject\FieldMission5\uwb\T005_uwb.db")

# Define a chunk size (number of rows to fetch at a time)
chunk_size = 10000000  # Adjust based on your memory capacity

# Initialize variables for iteration
offset = 0
chunk_index = 0
has_more_data = True

# Output directory
output_dir = r'Y:\Data\FieldProject\FieldMission5\uwb'

# Process data in chunks
while has_more_data:
    # Fetch a chunk of data from the database
    query = f"""
    SELECT shortid,
           timestamp,
           location_x, 
           location_y, 
           zones,
           alias,
           alternateid,
           groupnames
    FROM bravo_sql_databse_2024_CCVBehaviorTrial
    LIMIT {chunk_size} OFFSET {offset}
    """
    
    uwb = pd.read_sql_query(query, conn)
    
    if uwb.empty:
        has_more_data = False
    else:
        print(f"\nProcessing chunk {chunk_index} starting at offset {offset} with {len(uwb)} rows")

        # Initial row count
        initial_rows = len(uwb)

        # Create field time with ms
        uwb['field_time'] = pd.to_datetime(uwb['timestamp'], unit='ms', origin='unix', utc=True)

        # Adjust for timezone (e.g., UTC-4)
        uwb['field_time'] = uwb['field_time'] - pd.Timedelta(hours=4)

        # Remove timezone information after adjusting for -4 hours; keep field_time timezone naive
        uwb['field_time'] = uwb['field_time'].dt.tz_localize(None)

        # Define the origin point (start of the trial). First day of trial data lost due to me leaving the sql database open
        origin = pd.Timestamp('2024-06-22 12:00:00')

        # Calculate the noon_day column
        uwb['noon_day'] = np.ceil((uwb['field_time'] - origin) / np.timedelta64(1, 'D')).astype(int)

        # Convert location coordinates to meters from inches (from the wiser software)
        uwb['location_x'] *= 0.0254
        uwb['location_y'] *= 0.0254

        # Convert shortid to hex_id
        uwb['hex_id'] = uwb['shortid'].apply(lambda x: format(x, 'x'))

        # Merge on hex_id first, allowing multiple rows where hex_id matches; brings in both versions of the match with meta_code hex
        uwb = uwb.merge(meta_code_hex, on='hex_id', how='left') 

        # Filter rows based on the time interval; remove the rows where the uwb field_time doesn't fall in the meta_code_hex interval
        uwb = uwb[
            (uwb['field_time'] >= uwb['tag_start_time']) &
            ((uwb['tag_end_time'].isna()) | (uwb['field_time'] < uwb['tag_end_time']))
        ]

        # Count rows after interval filtering
        after_interval_filter = len(uwb)
        print(f"Rows after interval filtering: {after_interval_filter} (removed {initial_rows - after_interval_filter})")

        # Merge in metadata info and select final columns and order them as needed
        uwb = pd.merge(uwb, meta[['code', 'sex', 'trial']], on='code', how='left')
        uwb = uwb[['trial', 'sex', 'code', 'hex_id', 'noon_day', 'field_time', 'zones', 'location_x', 'location_y', 'alias', 'groupnames']]
        
        # FILTERING and smoothing on the chunk. 
        
        # Sort the data by animal and time to ensure proper calculation of velocity
        uwb = uwb.sort_values(by=['code', 'field_time'])

        # Calculate the time difference in seconds
        uwb['time_diff'] = uwb.groupby('code')['field_time'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds()

        # Calculate distances and velocities between consecutive points within each animal
        uwb['distance'] = np.sqrt((uwb['location_x'] - uwb.groupby('code')['location_x'].shift())**2 + 
                                  (uwb['location_y'] - uwb.groupby('code')['location_y'].shift())**2)
        uwb['velocity'] = uwb['distance'] / uwb['time_diff']

        # Additional filtering based on velocity, acceleration, and large jumps
        # Apply the velocity filter: remove any rows where the velocity exceeds a set threshold (e.g., 2 meters/second)
        velocity_threshold = 2  # meters/second
        uwb_before_velocity_filter = len(uwb)
        uwb = uwb[(uwb['velocity'] <= velocity_threshold) | (uwb['velocity'].isna())]
        after_velocity_filter = len(uwb)
        print(f"Rows after velocity filtering: {after_velocity_filter} (removed {uwb_before_velocity_filter - after_velocity_filter})")

        # Detect sudden jumps
        jump_threshold = 2  # in meters
        uwb['is_jump'] = (uwb['distance'] > jump_threshold)

        single_jumps = uwb['is_jump'].sum()
        print(f"Number of single jumps detected and removed: {single_jumps}")

        # Filter out these jumps
        uwb_before_jump_filter = len(uwb)
        uwb = uwb[~uwb['is_jump']]
        after_jump_filter = len(uwb)
        print(f"Rows after jump filtering: {after_jump_filter} (removed {uwb_before_jump_filter - after_jump_filter})")

        # Group consecutive points that fall within a set time window (in seconds)
        uwb['time_diff_s'] = np.ceil(uwb['time_diff']).astype(int)
        uwb['tw_group'] = uwb.groupby('code')['time_diff_s'].apply(lambda x: (x > 30).cumsum()).reset_index(level=0, drop=True)

        # Smoothing: Apply a 30-data point (i.e., rows) rolling average within each group; window sets points; min_periods of 1 means whatever data will be used if not enough
        uwb['smoothed_x'] = uwb.groupby(['code', 'tw_group'])['location_x'].transform(lambda x: x.rolling(window=30, min_periods=1).mean())
        uwb['smoothed_y'] = uwb.groupby(['code', 'tw_group'])['location_y'].transform(lambda y: y.rolling(window=30, min_periods=1).mean())

        # Downsample to 1Hz by taking the first point for each second for each animal
        # Create a new column `field_time_s` that truncates the `field_time` to seconds
        uwb['field_time_s'] = uwb['field_time'].dt.floor('s')

        # Ensure only one entry per code and field_time_s
        uwb_before_drop_duplicates = len(uwb)
        uwb = uwb.drop_duplicates(subset=['code', 'field_time_s']).reset_index(drop=True)
        after_drop_duplicates = len(uwb)
        print(f"Rows after dropping duplicates: {after_drop_duplicates} (removed {uwb_before_drop_duplicates - after_drop_duplicates})")

        final_rows = len(uwb)
        print(f"Final number of rows in this chunk: {final_rows}")

        uwb = uwb[['trial', 'sex', 'code', 'noon_day', 'field_time', 'smoothed_x', 'smoothed_y', 'location_x', 'location_y', 'zones', 'hex_id']]

        # Write the processed chunk to a separate CSV file
        chunk_output_file = os.path.join(output_dir, f'T005_uwb_1hz_chunk_{chunk_index}.csv')
        uwb.to_csv(chunk_output_file, index=False)
        
        # Increment the chunk index
        chunk_index += 1
    
    # Update offset to fetch the next chunk in the next iteration
    offset += chunk_size

# Close the connection
conn.close()

# List all chunk files
chunk_files = glob.glob(os.path.join(output_dir, 'T005_uwb_1hz_chunk_*.csv'))

# Read and concatenate all chunk files
all_data = pd.concat([pd.read_csv(f) for f in chunk_files])

# Write the concatenated data to the final output file
final_output_file = os.path.join(output_dir, 'T005_uwb_1hz.csv')
all_data.to_csv(final_output_file, index=False)

# Optionally, remove the chunk files to free up space
for f in chunk_files:
    os.remove(f)

print(f"All chunks have been concatenated and saved to {final_output_file}. Chunk files have been deleted.")


Processing chunk 0 starting at offset 0 with 10000000 rows
Rows after interval filtering: 8620913 (removed 1379087)
Rows after velocity filtering: 3479500 (removed 5141413)
Number of single jumps detected and removed: 141
Rows after jump filtering: 3479359 (removed 141)
Rows after dropping duplicates: 384246 (removed 3095113)
Final number of rows in this chunk: 384246

Processing chunk 1 starting at offset 10000000 with 10000000 rows
Rows after interval filtering: 6705576 (removed 3294424)
Rows after velocity filtering: 2832715 (removed 3872861)
Number of single jumps detected and removed: 23
Rows after jump filtering: 2832692 (removed 23)
Rows after dropping duplicates: 286282 (removed 2546410)
Final number of rows in this chunk: 286282

Processing chunk 2 starting at offset 20000000 with 10000000 rows
Rows after interval filtering: 10000000 (removed 0)
Rows after velocity filtering: 4163680 (removed 5836320)
Number of single jumps detected and removed: 16
Rows after jump filtering: 

C:\Users\ayalab\AppData\Local\Temp\ipykernel_1884\1964030606.py:160: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.concat([pd.read_csv(f) for f in chunk_files])
C:\Users\ayalab\AppData\Local\Temp\ipykernel_1884\1964030606.py:160: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.concat([pd.read_csv(f) for f in chunk_files])
C:\Users\ayalab\AppData\Local\Temp\ipykernel_1884\1964030606.py:160: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.concat([pd.read_csv(f) for f in chunk_files])
C:\Users\ayalab\AppData\Local\Temp\ipykernel_1884\1964030606.py:160: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.concat([pd.read_csv(f) for f in chunk_files])
C:\Users\ayalab\AppData\Local\Temp\ipykernel_1884\1964030606.py:160: Dty

All chunks have been concatenated and saved to Y:\Data\FieldProject\FieldMission5\uwb\T005_uwb_1hz.csv. Chunk files have been deleted.


In [ ]:
import pandas as pd
import numpy as np
import sqlite3

# Connect to the SQLite database, doesn't require loading the entire dataset into RAM. Database is approximately 84 million rows worth of data
conn = sqlite3.connect(r"Y:\Data\FieldProject\FieldMission5\uwb\T005_uwb.db")

# Define a chunk size (number of rows to fetch at a time)
chunk_size = 10000000  # Adjust based on your memory capacity

# Initialize variables for iteration
offset = 0
has_more_data = True

# Output file
output_file = r'Y:\Data\FieldProject\FieldMission5\uwb\T005_uwb_1hz.csv'

# Process data in chunks
while has_more_data:
    # Fetch a chunk of data from the database
    query = f"""
    SELECT shortid,
           timestamp,
           location_x, 
           location_y, 
           zones,
           alias,
           alternateid,
           groupnames
    FROM bravo_sql_databse_2024_CCVBehaviorTrial
    LIMIT {chunk_size} OFFSET {offset}
    """
    
    uwb = pd.read_sql_query(query, conn)
    
    if uwb.empty:
        has_more_data = False
    else:
        print(f"\nProcessing chunk starting at offset {offset} with {len(uwb)} rows")

        # Initial row count
        initial_rows = len(uwb)

        # Create field time with ms
        uwb['field_time'] = pd.to_datetime(uwb['timestamp'], unit='ms', origin='unix', utc=True)

        # Adjust for timezone (e.g., UTC-4)
        uwb['field_time'] = uwb['field_time'] - pd.Timedelta(hours=4)

        # Remove timezone information after adjusting for -4 hours; keep field_time timezone naive
        uwb['field_time'] = uwb['field_time'].dt.tz_localize(None)

        # Define the origin point (start of the trial). First day of trial data lost due to me leaving the sql database open
        origin = pd.Timestamp('2024-06-22 12:00:00')

        # Calculate the noon_day column
        uwb['noon_day'] = np.ceil((uwb['field_time'] - origin) / np.timedelta64(1, 'D')).astype(int)

        # Convert location coordinates to meters from inches (from the wiser software)
        uwb['location_x'] *= 0.0254
        uwb['location_y'] *= 0.0254

        # Convert shortid to hex_id
        uwb['hex_id'] = uwb['shortid'].apply(lambda x: format(x, 'x'))

        # Merge on hex_id first, allowing multiple rows where hex_id matches; brings in both versions of the match with meta_code hex
        uwb = uwb.merge(meta_code_hex, on='hex_id', how='left') 

        # Filter rows based on the time interval; remove the rows where the uwb field_time doesn't fall in the meta_code_hex interval
        uwb = uwb[
            (uwb['field_time'] >= uwb['tag_start_time']) &
            ((uwb['tag_end_time'].isna()) | (uwb['field_time'] < uwb['tag_end_time']))
        ]

        # Count rows after interval filtering
        after_interval_filter = len(uwb)
        print(f"Rows after interval filtering: {after_interval_filter} (removed {initial_rows - after_interval_filter})")

        # Merge in metadata info and select final columns and order them as needed
        uwb = pd.merge(uwb, meta[['code', 'sex', 'trial']], on='code', how='left')
        uwb = uwb[['trial', 'sex', 'code', 'hex_id', 'noon_day', 'field_time', 'zones', 'location_x', 'location_y', 'alias', 'groupnames']]
        
        # FILTERING and smoothing on the chunk. 
        
        # Sort the data by animal and time to ensure proper calculation of velocity
        uwb = uwb.sort_values(by=['code', 'field_time'])

        # Calculate the time difference in seconds
        uwb['time_diff'] = uwb.groupby('code')['field_time'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds()

        # Calculate distances and velocities between consecutive points within each animal
        uwb['distance'] = np.sqrt((uwb['location_x'] - uwb.groupby('code')['location_x'].shift())**2 + 
                                  (uwb['location_y'] - uwb.groupby('code')['location_y'].shift())**2)
        uwb['velocity'] = uwb['distance'] / uwb['time_diff']

        # Show descriptive statistics for velocity and distance before filtering
        # print("\nVelocity and Distance - Before Filtering")
        # print(uwb[['velocity', 'distance']].describe())

        # Additional filtering based on velocity, acceleration, and large jumps
        # Apply the velocity filter: remove any rows where the velocity exceeds a set threshold (e.g., 2 meters/second)
        velocity_threshold = 3  # meters/second
        uwb_before_velocity_filter = len(uwb)
        uwb = uwb[(uwb['velocity'] <= velocity_threshold) | (uwb['velocity'].isna())]
        after_velocity_filter = len(uwb)
        print(f"Rows after velocity filtering: {after_velocity_filter} (removed {uwb_before_velocity_filter - after_velocity_filter})")

        # Show descriptive statistics for velocity and distance after velocity filtering
        # print("\nVelocity and Distance - After Velocity Filtering")
        # print(uwb[['velocity', 'distance']].describe())

        # Detect sudden jumps
        jump_threshold = 2.5  # in meters
        uwb['is_jump'] = (uwb['distance'] > jump_threshold)

        single_jumps = uwb['is_jump'].sum()
        print(f"Number of single jumps detected and removed: {single_jumps}")

        # Filter out these jumps
        uwb_before_jump_filter = len(uwb)
        uwb = uwb[~uwb['is_jump']]
        after_jump_filter = len(uwb)
        print(f"Rows after jump filtering: {after_jump_filter} (removed {uwb_before_jump_filter - after_jump_filter})")

        # Show descriptive statistics for distance after jump filtering
        # print("\nDistance - After Jump Filtering")
        # print(uwb['distance'].describe())

        # round the time_diff to time_diff_s (seconds) for grouping procedure
        uwb['time_diff_s'] = np.ceil(uwb['time_diff']).astype(int)
        
        # Group consecutive points that fall within a set time window (in seconds)
        uwb['tw_group'] = uwb.groupby('code')['time_diff_s'].apply(lambda x: (x > 10).cumsum()).reset_index(level=0, drop=True)

        # Smoothing: Apply a 30-data point (i.e., rows) rolling average within each group; window sets points; min_periods of 1 means whatever data will be used if not enough
        uwb['smoothed_x'] = uwb.groupby(['code', 'tw_group'])['location_x'].transform(lambda x: x.rolling(window=30, min_periods=1).mean())
        uwb['smoothed_y'] = uwb.groupby(['code', 'tw_group'])['location_y'].transform(lambda y: y.rolling(window=30, min_periods=1).mean())

        # Downsample to 1Hz by taking the first point for each second for each animal
        # Create a new column `field_time_s` that truncates the `field_time` to seconds
        uwb['field_time_s'] = uwb['field_time'].dt.floor('s')
        
        # print(uwb.head())

        # Ensure only one entry per code and field_time_s
        uwb_before_drop_duplicates = len(uwb)
        uwb = uwb.drop_duplicates(subset=['code', 'field_time_s']).reset_index(drop=True)
        after_drop_duplicates = len(uwb)
        print(f"Rows after dropping duplicates: {after_drop_duplicates} (removed {uwb_before_drop_duplicates - after_drop_duplicates})")

        final_rows = len(uwb)
        print(f"Final number of rows in this chunk: {final_rows}")

        uwb = uwb[['trial', 'sex', 'code', 'noon_day', 'field_time', 'smoothed_x', 'smoothed_y', 'location_x', 'location_y', 'zones', 'hex_id']]

        # Append the processed chunk to the CSV file
        uwb.to_csv(output_file, mode='a', header=not offset, index=False)
    
    # Update offset to fetch the next chunk in the next iteration
    offset += chunk_size

# Close the connection
conn.close()


In [ ]:
import pandas as pd
import numpy as np
import sqlite3

# Connect to the SQLite database, doesn't require loading the entire dataset into RAM. Database is approximately 84 million rows worth of data
conn = sqlite3.connect(r"Y:\Data\FieldProject\FieldMission5\uwb\T005_uwb.db")

# Define a chunk size (number of rows to fetch at a time)
chunk_size = 10000000  # Adjust based on your memory capacity

# Initialize variables for iteration
offset = 0
has_more_data = True

# Output file
output_file = r'Y:\Data\FieldProject\FieldMission5\uwb\T005_uwb_1hz.csv'

# Process data in chunks
while has_more_data:
    # Fetch a chunk of data from the database
    query = f"""
    SELECT shortid,
           timestamp,
           location_x, 
           location_y, 
           zones,
           alias,
           alternateid,
           groupnames
    FROM bravo_sql_databse_2024_CCVBehaviorTrial
    ORDER BY shortid, timestamp
    LIMIT {chunk_size} OFFSET {offset}
    """
    
    uwb = pd.read_sql_query(query, conn)
    
    if uwb.empty:
        has_more_data = False
    else:
        print(f"\nProcessing chunk starting at offset {offset} with {len(uwb)} rows")

        # Initial row count
        initial_rows = len(uwb)

        # Create field time with ms
        uwb['field_time'] = pd.to_datetime(uwb['timestamp'], unit='ms', origin='unix', utc=True)

        # Adjust for timezone (e.g., UTC-4)
        uwb['field_time'] = uwb['field_time'] - pd.Timedelta(hours=4)

        # Remove timezone information after adjusting for -4 hours; keep field_time timezone naive
        uwb['field_time'] = uwb['field_time'].dt.tz_localize(None)

        # Define the origin point (start of the trial). First day of trial data lost due to me leaving the sql database open
        origin = pd.Timestamp('2024-06-22 12:00:00')

        # Calculate the noon_day column
        uwb['noon_day'] = np.ceil((uwb['field_time'] - origin) / np.timedelta64(1, 'D')).astype(int)

        # Convert location coordinates to meters from inches (from the wiser software)
        uwb['location_x'] *= 0.0254
        uwb['location_y'] *= 0.0254

        # Convert shortid to hex_id
        uwb['hex_id'] = uwb['shortid'].apply(lambda x: format(x, 'x'))

        # Merge on hex_id first, allowing multiple rows where hex_id matches; brings in both versions of the match with meta_code hex
        uwb = uwb.merge(meta_code_hex, on='hex_id', how='left') 

        # Filter rows based on the time interval; remove the rows where the uwb field_time doesn't fall in the meta_code_hex interval
        uwb = uwb[
            (uwb['field_time'] >= uwb['tag_start_time']) &
            ((uwb['tag_end_time'].isna()) | (uwb['field_time'] < uwb['tag_end_time']))
        ]

        # Count rows after interval filtering
        after_interval_filter = len(uwb)
        print(f"Rows after interval filtering: {after_interval_filter} (removed {initial_rows - after_interval_filter})")

        # Merge in metadata info and select final columns and order them as needed
        uwb = pd.merge(uwb, meta[['code', 'sex', 'trial']], on='code', how='left')

        uwb = uwb[['trial', 'sex', 'code', 'hex_id', 'noon_day', 'field_time', 'zones', 'location_x', 'location_y', 'alias', 'groupnames']]

        # FILTERING and smoothing on the chunk. 
        
        # Sort the data by animal and time to ensure proper calculation of velocity
        uwb = uwb.sort_values(by=['code', 'field_time'])

        # Calculate the time difference in seconds
        uwb['time_diff'] = uwb.groupby('code')['field_time'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds()

        # Calculate distances and velocities between consecutive points within each animal
        uwb['distance'] = np.sqrt((uwb['location_x'] - uwb.groupby('code')['location_x'].shift())**2 + 
                                  (uwb['location_y'] - uwb.groupby('code')['location_y'].shift())**2)
        uwb['velocity'] = uwb['distance'] / uwb['time_diff']

        # Show descriptive statistics for velocity and distance before filtering
        print("\nVelocity and Distance - Before Filtering")
        print(uwb[['velocity', 'distance']].describe())

        # Additional filtering based on velocity, acceleration, and large jumps
        # Apply the velocity filter: remove any rows where the velocity exceeds a set threshold (e.g., 2 meters/second)
        velocity_threshold = 3  # meters/second
        uwb_before_velocity_filter = len(uwb)
        uwb = uwb[(uwb['velocity'] <= velocity_threshold) | (uwb['velocity'].isna())]
        after_velocity_filter = len(uwb)
        print(f"Rows after velocity filtering: {after_velocity_filter} (removed {uwb_before_velocity_filter - after_velocity_filter})")

        # Show descriptive statistics for velocity and distance after velocity filtering
        print("\nVelocity and Distance - After Velocity Filtering")
        print(uwb[['velocity', 'distance']].describe())

        # Detect sudden jumps
        jump_threshold = 2.5  # in meters
        uwb['is_jump'] = (uwb['distance'] > jump_threshold)

        single_jumps = uwb['is_jump'].sum()
        print(f"Number of single jumps detected and removed: {single_jumps}")

        # Filter out these jumps
        uwb_before_jump_filter = len(uwb)
        uwb = uwb[~uwb['is_jump']]
        after_jump_filter = len(uwb)
        print(f"Rows after jump filtering: {after_jump_filter} (removed {uwb_before_jump_filter - after_jump_filter})")

        # Show descriptive statistics for distance after jump filtering
        print("\nDistance - After Jump Filtering")
        print(uwb['distance'].describe())

        # round the time_diff to time_diff_s (seconds) for grouping procedure
        uwb['time_diff_s'] = np.ceil(uwb['time_diff']).astype(int)
        
        # Group consecutive points that fall within a set time window (in seconds)
        uwb['tw_group'] = uwb.groupby('code')['time_diff_s'].apply(lambda x: (x > 10).cumsum()).reset_index(level=0, drop=True)

        # Smoothing: Apply a 30-data point (i.e., rows) rolling average within each group; window sets points; min_periods of 1 means whatever data will be used if not enough
        uwb['smoothed_x'] = uwb.groupby(['code', 'tw_group'])['location_x'].transform(lambda x: x.rolling(window=30, min_periods=1).mean())
        uwb['smoothed_y'] = uwb.groupby(['code', 'tw_group'])['location_y'].transform(lambda y: y.rolling(window=30, min_periods=1).mean())

        # Ensure uniqueness by dropping duplicates on the field_time for each code
        uwb_before_drop_duplicates = len(uwb)
        uwb = uwb.drop_duplicates(subset=['code', 'field_time'])
        after_drop_duplicates = len(uwb)
        print(f"Rows after dropping duplicates: {after_drop_duplicates} (removed {uwb_before_drop_duplicates - after_drop_duplicates})")

        # Downsample to 1Hz by taking the first point for each second for each animal
        uwb = uwb.groupby(['code', uwb['field_time'].dt.floor('s')]).first().reset_index(level=1, drop=True).reset_index()

        final_rows = len(uwb)
        print(f"Final number of rows in this chunk: {final_rows}")

        uwb = uwb[['trial', 'sex', 'code', 'noon_day', 'field_time', 'smoothed_x', 'smoothed_y', 'location_x', 'location_y', 'zones', 'hex_id', 'alias', 'groupnames']]

        # Append the processed chunk to the CSV file
        uwb.to_csv(output_file, mode='a', header=not offset, index=False)
    
    # Update offset to fetch the next chunk in the next iteration
    offset += chunk_size

# Close the connection
conn.close()


In [ ]:
output_fp = r"Y:\Data\FieldProject\Output"
df = pd.read_csv(r'Y:\Data\FieldProject\FieldMission5\uwb\T005_uwb_1hz.csv')  # Adjust the path to your metadata file
len(df)
df.head()


In [ ]:
import pandas as pd
import numpy as np
import sqlite3

# Connect to the SQLite database, doesn't require loading the entire dataset into RAM. Database is approximately 84 million rows worth of data
conn = sqlite3.connect(r"Y:\Data\FieldProject\FieldMission5\uwb\T005_uwb.db")

# Define a chunk size (number of rows to fetch at a time)
chunk_size = 10000000  # Adjust based on your memory capacity

# Initialize variables for iteration
offset = 0
has_more_data = True

# Output file
output_file = r'Y:\Data\FieldProject\FieldMission5\uwb\T005_uwb_1hz.csv'

# Process data in chunks
while has_more_data:
    # Fetch a chunk of data from the database
    query = f"""
    SELECT shortid,
           timestamp,
           location_x, 
           location_y, 
           zones,
           alias,
           alternateid,
           groupnames
    FROM bravo_sql_databse_2024_CCVBehaviorTrial
    ORDER BY shortid, timestamp
    LIMIT {chunk_size} OFFSET {offset}
    """
    
    uwb = pd.read_sql_query(query, conn)
    
    if uwb.empty:
        has_more_data = False
    else:
        print(f"\nProcessing chunk starting at offset {offset} with {len(uwb)} rows")

        # Initial row count
        initial_rows = len(uwb)

        # Create field time with ms
        uwb['field_time'] = pd.to_datetime(uwb['timestamp'], unit='ms', origin='unix', utc=True)

        # Adjust for timezone (e.g., UTC-4)
        uwb['field_time'] = uwb['field_time'] - pd.Timedelta(hours=4)

        # Remove timezone information after adjusting for -4 hours; keep field_time timezone naive
        uwb['field_time'] = uwb['field_time'].dt.tz_localize(None)

        # Define the origin point (start of the trial). First day of trial data lost due to me leaving the sql database open
        origin = pd.Timestamp('2024-06-22 12:00:00')

        # Calculate the noon_day column
        uwb['noon_day'] = np.ceil((uwb['field_time'] - origin) / np.timedelta64(1, 'D')).astype(int)

        # Convert location coordinates to meters from inches (from the wiser software)
        uwb['location_x'] *= 0.0254
        uwb['location_y'] *= 0.0254

        # Convert shortid to hex_id
        uwb['hex_id'] = uwb['shortid'].apply(lambda x: format(x, 'x'))

        # Merge on hex_id first, allowing multiple rows where hex_id matches; brings in both versions of the match with meta_code hex
        uwb = uwb.merge(meta_code_hex, on='hex_id', how='left') 

        # Filter rows based on the time interval; remove the rows where the uwb field_time doesn't fall in the meta_code_hex interval
        uwb = uwb[
            (uwb['field_time'] >= uwb['tag_start_time']) &
            ((uwb['tag_end_time'].isna()) | (uwb['field_time'] < uwb['tag_end_time']))
        ]

        # Count rows after interval filtering
        after_interval_filter = len(uwb)
        print(f"Rows after interval filtering: {after_interval_filter} (removed {initial_rows - after_interval_filter})")

        # Merge in metadata info and select final columns and order them as needed
        uwb = pd.merge(uwb, meta[['code', 'sex', 'trial']], on='code', how='left')

        uwb = uwb[['trial', 'sex', 'code', 'hex_id', 'noon_day', 'field_time', 'zones', 'location_x', 'location_y', 'alias', 'groupnames']]

        # FILTERING and smoothing on the chunk. 
        
        # Sort the data by animal and time to ensure proper calculation of velocity
        uwb = uwb.sort_values(by=['code', 'field_time'])

        # Calculate the time difference in seconds
        uwb['time_diff'] = uwb.groupby('code')['field_time'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds()

        # Calculate distances and velocities between consecutive points within each animal
        uwb['distance'] = np.sqrt((uwb['location_x'] - uwb.groupby('code')['location_x'].shift())**2 + 
                                  (uwb['location_y'] - uwb.groupby('code')['location_y'].shift())**2)
        uwb['velocity'] = uwb['distance'] / uwb['time_diff']

        # Additional filtering based on velocity, acceleration, and large jumps
        # Apply the velocity filter: remove any rows where the velocity exceeds a set threshold (e.g., 2 meters/second)
        velocity_threshold = 3  # meters/second
        uwb_before_velocity_filter = len(uwb)
        uwb = uwb[(uwb['velocity'] <= velocity_threshold) | (uwb['velocity'].isna())]
        after_velocity_filter = len(uwb)
        print(f"Rows after velocity filtering: {after_velocity_filter} (removed {uwb_before_velocity_filter - after_velocity_filter})")

        # Detect sudden jumps
        jump_threshold = 2.5  # in meters
        uwb['is_jump'] = (uwb['distance'] > jump_threshold)

        single_jumps = uwb['is_jump'].sum()
        print(f"Number of single jumps detected and removed: {single_jumps}")

        # Filter out these jumps
        uwb_before_jump_filter = len(uwb)
        uwb = uwb[~uwb['is_jump']]
        after_jump_filter = len(uwb)
        print(f"Rows after jump filtering: {after_jump_filter} (removed {uwb_before_jump_filter - after_jump_filter})")

        # round the time_diff to time_diff_s (seconds) for grouping procedure
        uwb['time_diff_s'] = np.ceil(uwb['time_diff']).astype(int)
        
        # Group consecutive points that fall within a set time window (in seconds)
        uwb['tw_group'] = uwb.groupby('code')['time_diff_s'].apply(lambda x: (x > 10).cumsum()).reset_index(level=0, drop=True)

        # Smoothing: Apply a 30-data point (i.e., rows) rolling average within each group; window sets points; min_periods of 1 means whatever data will be used if not enough
        uwb['smoothed_x'] = uwb.groupby(['code', 'tw_group'])['location_x'].transform(lambda x: x.rolling(window=30, min_periods=1).mean())
        uwb['smoothed_y'] = uwb.groupby(['code', 'tw_group'])['location_y'].transform(lambda y: y.rolling(window=30, min_periods=1).mean())

        # Ensure uniqueness by dropping duplicates on the field_time for each code
        uwb_before_drop_duplicates = len(uwb)
        uwb = uwb.drop_duplicates(subset=['code', 'field_time'])
        after_drop_duplicates = len(uwb)
        print(f"Rows after dropping duplicates: {after_drop_duplicates} (removed {uwb_before_drop_duplicates - after_drop_duplicates})")

        # Downsample to 1Hz by taking the first point for each second for each animal
        uwb = uwb.groupby(['code', uwb['field_time'].dt.floor('s')]).first().reset_index(level=1, drop=True).reset_index()

        final_rows = len(uwb)
        print(f"Final number of rows in this chunk: {final_rows}")

        uwb = uwb[['trial', 'sex', 'code', 'noon_day', 'field_time', 'smoothed_x', 'smoothed_y', 'location_x', 'location_y', 'zones', 'hex_id', 'alias', 'groupnames']]

        # Append the processed chunk to the CSV file
        uwb.to_csv(output_file, mode='a', header=not offset, index=False)
    
    # Update offset to fetch the next chunk in the next iteration
    offset += chunk_size

# Close the connection
conn.close()

In [ ]:
import pandas as pd
import numpy as np
import sqlite3

# Connect to the SQLite database, doesn't require loading the entire dataset into RAM. Database is approximately 84 million rows worth of data
conn = sqlite3.connect(r"Y:\Data\FieldProject\FieldMission5\uwb\T005_uwb.db")

# Define a chunk size (number of rows to fetch at a time)
chunk_size = 10000000  # Adjust based on your memory capacity

# Initialize variables for iteration
offset = 0
has_more_data = True

# Output file
output_file = r'Y:\Data\FieldProject\FieldMission5\uwb\T005_uwb_1hz.csv'

# Process data in chunks
while has_more_data:
    # Fetch a chunk of data from the database
    query = f"""
    SELECT shortid,
           timestamp,
           location_x, 
           location_y, 
           zones,
           alias,
           alternateid,
           groupnames
    FROM bravo_sql_databse_2024_CCVBehaviorTrial
    ORDER BY shortid, timestamp
    LIMIT {chunk_size} OFFSET {offset}
    """
    
    uwb = pd.read_sql_query(query, conn)
    
    if uwb.empty:
        has_more_data = False
    else:
        # Perform your data processing on `uwb` here

        # Create field time with ms
        uwb['field_time'] = pd.to_datetime(uwb['timestamp'], unit='ms', origin='unix', utc=True)

        # Adjust for timezone (e.g., UTC-4)
        uwb['field_time'] = uwb['field_time'] - pd.Timedelta(hours=4)

        # Remove timezone information after adjusting for -4 hours; keep field_time timezone naive
        uwb['field_time'] = uwb['field_time'].dt.tz_localize(None)

        # Define the origin point (start of the trial). First day of trial data lost due to me leaving the sql database open
        origin = pd.Timestamp('2024-06-22 12:00:00')

        # Calculate the noon_day column
        uwb['noon_day'] = np.ceil((uwb['field_time'] - origin) / np.timedelta64(1, 'D')).astype(int)

        # Convert location coordinates to meters from inches (from the wiser software)
        uwb['location_x'] *= 0.0254
        uwb['location_y'] *= 0.0254

        # Convert shortid to hex_id
        uwb['hex_id'] = uwb['shortid'].apply(lambda x: format(x, 'x'))

        # Merge on hex_id first, allowing multiple rows where hex_id matches; brings in both versions of the match with meta_code hex
        uwb = uwb.merge(meta_code_hex, on='hex_id', how='left') 

        # Filter rows based on the time interval; remove the rows where the uwb field_time doesn't fall in the meta_code_hex interval
        uwb = uwb[
            (uwb['field_time'] >= uwb['tag_start_time']) &
            ((uwb['tag_end_time'].isna()) | (uwb['field_time'] < uwb['tag_end_time']))
        ]

        # Merge in metadata info and select final columns and order them as needed
        uwb = pd.merge(uwb, meta[['code', 'sex', 'trial']], on='code', how='left')

        uwb = uwb[['trial', 'sex', 'code', 'hex_id', 'noon_day', 'field_time', 'zones', 'location_x', 'location_y', 'alias', 'groupnames']]

        # FILTERING and smoothing on the chunk. 
        
        # Sort the data by animal and time to ensure proper calculation of velocity
        uwb = uwb.sort_values(by=['code', 'field_time'])

        # Calculate the time difference in seconds
        uwb['time_diff'] = uwb.groupby('code')['field_time'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds()

        # Calculate distances and velocities between consecutive points within each animal
        uwb['distance'] = np.sqrt((uwb['location_x'] - uwb.groupby('code')['location_x'].shift())**2 + 
                                  (uwb['location_y'] - uwb.groupby('code')['location_y'].shift())**2)
        uwb['velocity'] = uwb['distance'] / uwb['time_diff']

        # Additional filtering based on velocity, acceleration, and large jumps
        # Apply the velocity filter: remove any rows where the velocity exceeds a set threshold (e.g., 2 meters/second)
        velocity_threshold = 3  # meters/second
        uwb = uwb[(uwb['velocity'] <= velocity_threshold) | (uwb['velocity'].isna())]

        # Detect sudden jumps
        jump_threshold = 2.5  # in meters
        uwb['is_jump'] = (uwb['distance'] > jump_threshold)

        single_jumps = uwb['is_jump'].sum()
        print(f"Number of single jumps removed: {single_jumps}")

        # Filter out these jumps
        uwb = uwb[~uwb['is_jump']]

        # round the time_diff to time_diff_s (seconds) for grouping procedure
        uwb['time_diff_s'] = np.ceil(uwb['time_diff']).astype(int)
        
        # Group consecutive points that fall within a set time window (in seconds)
        uwb['tw_group'] = uwb.groupby('code')['time_diff_s'].apply(lambda x: (x > 10).cumsum()).reset_index(level=0, drop=True)

        # Smoothing: Apply a 30-data point (i.e., rows) rolling average within each group; window sets points; min_periods of 1 means whatever data will be used if not enough
        uwb['smoothed_x'] = uwb.groupby(['code', 'tw_group'])['location_x'].transform(lambda x: x.rolling(window=30, min_periods=1).mean())
        uwb['smoothed_y'] = uwb.groupby(['code', 'tw_group'])['location_y'].transform(lambda y: y.rolling(window=30, min_periods=1).mean())

        # Downsample to 1Hz by taking the first point for each second for each animal
        uwb = uwb.groupby(['code', uwb['field_time'].dt.floor('s')]).first().reset_index(level=1, drop=True).reset_index()

        uwb = uwb[['trial','sex','code','noon_day','field_time','smoothed_x','smoothed_y','location_x','location_y','zones','hex_id','alias','groupnames']]

        # Append the processed chunk to the CSV file
        uwb.to_csv(output_file, mode='a', header=not offset, index=False)
    
    # Update offset to fetch the next chunk in the next iteration
    offset += chunk_size

# Close the connection
conn.close()

# Single chunk testing

In [ ]:
## single chunk testing script. 

import pandas as pd
import numpy as np
import sqlite3

# Connect to the SQLite database, doesn't require loading the entire dataset into RAM. Database is approximately 84 million rows worth of data
conn = sqlite3.connect(r"Y:\Data\FieldProject\FieldMission5\uwb\T005_uwb.db")

# Define a chunk size (number of rows to fetch at a time)
chunk_size = 50000000  # Adjust based on your memory capacity

# Initialize variables for iteration
offset = 0
has_more_data = True

# Perform a single iteration for testing
query = f"""
SELECT shortid,
       timestamp,
       location_x, 
       location_y, 
       zones,
       alias,
       alternateid,
       groupnames
FROM bravo_sql_databse_2024_CCVBehaviorTrial
ORDER BY shortid, timestamp
LIMIT {chunk_size} OFFSET {offset}
"""

# Read a chunk of data into a DataFrame
uwb = pd.read_sql_query(query, conn)

if uwb.empty:
    has_more_data = False
else:
    # Perform your data processing on `uwb` here

    # Create field time with ms
    uwb['field_time'] = pd.to_datetime(uwb['timestamp'], unit='ms', origin='unix', utc=True)

    # Adjust for timezone (e.g., UTC-4)
    uwb['field_time'] = uwb['field_time'] - pd.Timedelta(hours=4)

    # Remove timezone information after adjusting for -4 hours; keep field_time timezone naive
    uwb['field_time'] = uwb['field_time'].dt.tz_localize(None)

    # Define the origin point (start of the trial). First day of trial data lost due to me leaving the sql database open
    origin = pd.Timestamp('2024-06-22 12:00:00')

    # Calculate the noon_day column
    uwb['noon_day'] = np.ceil((uwb['field_time'] - origin) / np.timedelta64(1, 'D')).astype(int)

    # Convert location coordinates to meters from inches (from the wiser software)
    uwb['location_x'] *= 0.0254
    uwb['location_y'] *= 0.0254

    # Convert shortid to hex_id
    uwb['hex_id'] = uwb['shortid'].apply(lambda x: format(x, 'x'))

    # Merge on hex_id first, allowing multiple rows where hex_id matches; brings in both versions of the match with meta_code hex
    uwb = uwb.merge(meta_code_hex, on='hex_id', how='left') 

    # Filter rows based on the time interval; remove the rows where the uwb field_time doesn't fall in the meta_code_hex interval
    uwb = uwb[
        (uwb['field_time'] >= uwb['tag_start_time']) &
        ((uwb['tag_end_time'].isna()) | (uwb['field_time'] < uwb['tag_end_time']))
    ]

    # Merge in metadata info and select final columns and order them as needed
    uwb = pd.merge(uwb, meta[['code', 'sex', 'trial']], on='code', how='left')

    uwb = uwb[['trial', 'sex', 'code', 'hex_id', 'noon_day', 'field_time', 'zones', 'location_x', 'location_y', 'alias', 'groupnames']]

    # FILTERING and smoothing on the chunk. 
    
    # Sort the data by animal and time to ensure proper calculation of velocity
    uwb = uwb.sort_values(by=['code', 'field_time'])

    # Calculate the time difference in seconds
    uwb['time_diff'] = uwb.groupby('code')['field_time'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds()

    # Calculate distances and velocities between consecutive points within each animal
    uwb['distance'] = np.sqrt((uwb['location_x'] - uwb.groupby('code')['location_x'].shift())**2 + 
                              (uwb['location_y'] - uwb.groupby('code')['location_y'].shift())**2)
    uwb['velocity'] = uwb['distance'] / uwb['time_diff']

    # Additional filtering based on velocity, acceleration, and large jumps
    # Apply the velocity filter: remove any rows where the velocity exceeds a set threshold (e.g., 2 meters/second)
    velocity_threshold = 3  # meters/second
    uwb = uwb[(uwb['velocity'] <= velocity_threshold) | (uwb['velocity'].isna())]

    # Detect sudden jumps
    jump_threshold = 2.5  # in meters
    uwb['is_jump'] = (uwb['distance'] > jump_threshold)

    single_jumps = uwb['is_jump'].sum()
    print(f"Number of single jumps removed: {single_jumps}")

    # Filter out these jumps
    uwb = uwb[~uwb['is_jump']]

    # round the time_diff to time_diff_s (seconds) for grouping procedure
    uwb['time_diff_s'] = np.ceil(uwb['time_diff']).astype(int)
    
    # Group consecutive points that fall within a set time window (in seconds)
    uwb['tw_group'] = uwb.groupby('code')['time_diff_s'].apply(lambda x: (x > 10).cumsum()).reset_index(level=0, drop=True)

    # Smoothing: Apply a 30-data point (i.e., rows) rolling average within each group; window sets points; min_periods of 1 means whatever data will be used if not enough
    uwb['smoothed_x'] = uwb.groupby(['code', 'tw_group'])['location_x'].transform(lambda x: x.rolling(window=30, min_periods=1).mean())
    uwb['smoothed_y'] = uwb.groupby(['code', 'tw_group'])['location_y'].transform(lambda y: y.rolling(window=30, min_periods=1).mean())

    # Downsample to 1Hz by taking the first point for each second for each animal
    uwb = uwb.groupby(['code', uwb['field_time'].dt.floor('S')]).first().reset_index(level=1, drop=True).reset_index()

    uwb = uwb[['trial','sex','code','noon_day','field_time','smoothed_x','smoothed_y','location_x','location_y','zones','hex_id','alias','groupnames']]

    # Display the first few rows to verify
    print(uwb.head())

    # (Optionally) Write the first chunk to a CSV to verify
    uwb.to_csv(r'Y:\Data\FieldProject\FieldMission5\uwb\T005_uwb_1hz.csv', index=False)

# Close the connection
conn.close()


# old approach

In [ ]:
## OLD APPROACH

# Define your SQL query
query = """
SELECT shortid, 
       calculation_error,
       location_x, 
       location_y, 
       anchors_used,
       battery_voltage,
       zones,
       alias,
       alternateid,
       groupnames,
       CAST(timestamp / 1000 AS INTEGER) as timestamp_sec, # this removes the ms values
       MIN(timestamp) as original_timestamp
FROM bravo_sql_databse_2024_CCVBehaviorTrial
GROUP BY shortid, timestamp_sec
ORDER BY shortid, timestamp_sec
"""

# Execute the query and write the results directly to a CSV file
output_file = r'Y:\Data\FieldProject\FieldMission5\uwb\T005_uwb_1hz.csv'

with open(output_file, 'w', newline='') as f:
    writer = csv.writer(f)
    cursor = conn.cursor()
    
    # Execute the query
    cursor.execute(query)
    
    # Write the header
    writer.writerow([i[0] for i in cursor.description])
    
    # Write the rows
    writer.writerows(cursor.fetchall())

# Close the connection
conn.close()

print(f"Data has been exported to {output_file}")

Now read in the 1hz data file and merge in the most important metadata information. 

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

uwb = pd.read_csv(r"Y:\Data\FieldProject\FieldMission5\uwb\uwb_1hz.csv")

# Create field time with ms
uwb['field_time'] = pd.to_datetime(uwb['original_timestamp'], unit='ms', origin='unix', utc=True)

# Adjust for timezone (e.g., UTC-4)
uwb['field_time'] = uwb['field_time'] - pd.Timedelta(hours=4)

# Remove timezone information after adjusting for -4 hours; keep field_time timezone naive
uwb['field_time'] = uwb['field_time'].dt.tz_localize(None)

# Define the origin point (start of the trial). First day of trial data lost due to me leaving the sql database open
origin = pd.Timestamp('2024-06-22 12:00:00')

# Calculate the noon_day column
uwb['noon_day'] = np.ceil((uwb['field_time'] - origin) / np.timedelta64(1, 'D')).astype(int)

# Convert location coordinates to meters from inches (from the wiser software)
uwb['location_x'] *= 0.0254
uwb['location_y'] *= 0.0254

# Convert shortid to hex_id
uwb['hex_id'] = uwb['shortid'].apply(lambda x: format(x, 'x'))

# Merge on hex_id first, allowing multiple rows where hex_id matches; brings in both versions of the match with meta_code hex
merged = uwb.merge(meta_code_hex, on='hex_id', how='left') 

# Filter rows based on the time interval; remove the rows where the uwb field_time doesnt fall in the meta_code_hex interval
filtered = merged[
    (merged['field_time'] >= merged['tag_start_time']) &
    ((merged['tag_end_time'].isna()) | (merged['field_time'] < merged['tag_end_time']))
]

## Note: Need to come back and confirm that the grouping by code here is correct
uwb_filtered = filtered.copy()

# Merge in metadata info and select final columns and order them as needed

# Merge uwb_filtered with the metadata sheet on the 'code' column
uwb_merged = pd.merge(uwb_filtered, meta[['code', 'sex', 'trial']], on='code', how='left')

# Display the merged DataFrame to verify
# uwb_merged.head()

uwb_filtered = uwb_merged[['trial', 'sex', 'code', 'hex_id', 'noon_day', 'field_time', 'zones',
                            'location_x', 'location_y', 'alias', 'groupnames', 'original_timestamp', 
                            'anchors_used', 'calculation_error', 'battery_voltage']]


# Display the first few rows to verify
# uwb_filtered.head()

uwb_filtered.to_csv(r'Y:\Data\FieldProject\Results\T005_uwb_1hz_clean.csv',index=False)